These reuters and imdb dataset are abit tricky to minpulate the data and hard to train with lstm, gru and bilateral lstm models
I think that it is becuase the input size are too heavy and big it almost take 30 min and more ram with bilateral lstm models.
I think I have to try with transformer model and also I need take and try with plain text dataset.

In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np 
from tensorflow.keras import layers
from tensorflow import keras
from datetime import datetime
import pandas as pd

In [3]:

tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)
tf.set_seed = 42
epoch = 10
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 600

In [4]:
# import data
(train_features,train_labels), (test_features, test_labels) = tf.keras.datasets.reuters.load_data(num_words=max_vocab_length)

# Check the data shape
print(
    f"Train_features : {train_features.shape} {train_features.dtype} \n" 
    f"Train_labels : {train_labels.shape} {train_labels.dtype} \n" 
    f"Test features : {test_features.shape} {test_features.dtype} \n" 
    f"Test_labels : {test_labels.shape} {test_labels.dtype} "
    )

Train_features : (8982,) object 
Train_labels : (8982,) int64 
Test features : (2246,) object 
Test_labels : (2246,) int64 


In [5]:
tensor_test = tf.cast(train_features[1], dtype=tf.float32)

print(tensor_test)

tf.Tensor(
[1.000e+00 3.267e+03 6.990e+02 3.434e+03 2.295e+03 5.600e+01 2.000e+00
 7.511e+03 9.000e+00 5.600e+01 3.906e+03 1.073e+03 8.100e+01 5.000e+00
 1.198e+03 5.700e+01 3.660e+02 7.370e+02 1.320e+02 2.000e+01 4.093e+03
 7.000e+00 2.000e+00 4.900e+01 2.295e+03 2.000e+00 1.037e+03 3.267e+03
 6.990e+02 3.434e+03 8.000e+00 7.000e+00 1.000e+01 2.410e+02 1.600e+01
 8.550e+02 1.290e+02 2.310e+02 7.830e+02 5.000e+00 4.000e+00 5.870e+02
 2.295e+03 2.000e+00 2.000e+00 7.750e+02 7.000e+00 4.800e+01 3.400e+01
 1.910e+02 4.400e+01 3.500e+01 1.795e+03 5.050e+02 1.700e+01 1.200e+01], shape=(56,), dtype=float32)


In [6]:
# VECTORIZE function

def vectorize_sequences(sequences, dimension=max_vocab_length):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return tf.cast(results, dtype= tf.float32)


In [7]:
# Vectorize and Normalize train and test to tensors with 10k columns

train_features_tf = vectorize_sequences(train_features)
test_features_tf = vectorize_sequences(test_features)

print("train features ", train_features_tf.shape)
print("test features ", test_features_tf.shape)

train features  (8982, 10000)
test features  (2246, 10000)


In [8]:
# ONE HOT ENCODER of the labels

one_hot_train_labels = tf.keras.utils.to_categorical(train_labels)
one_hot_test_labels = tf.keras.utils.to_categorical(test_labels)

print("one_hot_train_labels ", one_hot_train_labels.shape)
print("one_hot_test_labels ", one_hot_test_labels.shape)



one_hot_train_labels  (8982, 46)
one_hot_test_labels  (2246, 46)


In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_features_tf, one_hot_train_labels))
train_dataset =  train_dataset.shuffle(8982).batch(128).cache().prefetch(tf.data.AUTOTUNE)
valid_dataset = tf.data.Dataset.from_tensor_slices((test_features_tf,one_hot_test_labels))
valid_dataset = valid_dataset.batch(128).cache().prefetch(tf.data.AUTOTUNE)
print(f"Train : {train_dataset} \n"
      f"Test : {valid_dataset}")

Train : <PrefetchDataset element_spec=(TensorSpec(shape=(None, 10000), dtype=tf.float32, name=None), TensorSpec(shape=(None, 46), dtype=tf.float32, name=None))> 
Test : <PrefetchDataset element_spec=(TensorSpec(shape=(None, 10000), dtype=tf.float32, name=None), TensorSpec(shape=(None, 46), dtype=tf.float32, name=None))>


In [10]:
# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss", # watch the val loss metric
                                                  patience=5) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",  
                                                 factor=0.2, # multiply the learning rate by 0.2 (reduce by 5x)
                                                 patience=3,
                                                 verbose=1, # print out when learning rate goes down 
                                                 min_lr=1e-7)

In [11]:
# Set random seed and create embedding layer (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
embedding_layers = layers.Embedding(input_dim=max_vocab_length,
                                     output_dim=2,
                                     embeddings_initializer="uniform",
                                     input_length = max_length,
                                     name="embedding_layers")

Model 1

In [12]:
# Build a Bidirectional RNN in TensorFlow
inputs = layers.Input(shape=(max_vocab_length,))
x = embedding_layers(inputs)
x = layers.Bidirectional(layers.LSTM(64))(x) # stacking RNN layers requires return_sequences=True
#x = layers.Bidirectional(layers.LSTM(64))(x) # bidirectional goes both ways so has double the parameters of a regular LSTM layer
outputs = layers.Dense(46, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

# Compile
model_1.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary of our bidirectional model
model_1.summary()

# Fit the model (takes longer because of the bidirectional layers)
#model_1_history = model_1.fit(train_dataset,
#                           epochs=5,
#                           validation_data=valid_dataset,
#                           callbacks=[early_stopping, reduce_lr])

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10000)]           0         
                                                                 
 embedding_layers (Embedding  (None, 10000, 2)         20000     
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 128)              34304     
 l)                                                              
                                                                 
 dense (Dense)               (None, 46)                5934      
                                                                 
Total params: 60,238
Trainable params: 60,238
Non-trainable params: 0
_________________________________________________________________


In [13]:
inputs = layers.Input(shape=(max_vocab_length,))
x = embedding_layers(inputs)
x = layers.Conv1D(32, kernel_size=5, padding="same", activation="relu")(x)
x = layers.Conv1D(64, kernel_size=5, padding="same", activation="relu")(x)
x = layers.Conv1D(128, kernel_size=5, padding="same", activation="relu")(x)
x = layers.Flatten()(x) # condense the output of our feature vector
outputs = layers.Dense(46, activation="softmax")(x)
model_1= tf.keras.Model(inputs, outputs, name="model_4_Bidirectional")

# Compile
model_1.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1.summary()

# start = datetime.now()
# model_1_history = model_1.fit(train_dataset,
#                            epochs=5,
#                            validation_data=valid_dataset,
#                            callbacks=[early_stopping, reduce_lr])

# end = datetime.now()
# print(f"The time taken to fit the modle is {end - start}")
# model_1.evaluate(valid_dataset)

Model: "model_4_Bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 10000)]           0         
                                                                 
 embedding_layers (Embedding  (None, 10000, 2)         20000     
 )                                                               
                                                                 
 conv1d (Conv1D)             (None, 10000, 32)         352       
                                                                 
 conv1d_1 (Conv1D)           (None, 10000, 64)         10304     
                                                                 
 conv1d_2 (Conv1D)           (None, 10000, 128)        41088     
                                                                 
 flatten (Flatten)           (None, 1280000)           0         
                                             

Hub and USE emcode layers

In [14]:
# import data
(train_features,train_labels), (test_features, test_labels) = tf.keras.datasets.reuters.load_data()

In [15]:
word_index = tf.keras.datasets.reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
len(reverse_word_index)


30979

In [32]:
def decode_tokenized_sentence(sentences_array):
    decode_sent = []
    for values in sentences_array:
        decoded_newswire = [' '.join([reverse_word_index.get( i - 3, '?') for i in values ]) ]
        decode_sent.append(decoded_newswire)
    return tf.constant(tf.squeeze(decode_sent))

In [33]:
train_decode = decode_tokenized_sentence(train_features)
test_decode = decode_tokenized_sentence(test_features)
train_decode.shape, test_decode.shape

(TensorShape([8982]), TensorShape([2246]))

In [34]:
import tensorflow_hub as hub
# Set random seed and create embedding layer (new embedding layer for each model)
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE")

sentence_encoder_layer

In [35]:
#BUild USE model 
# inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
# x = sentence_encoder_layer(inputs)
# x = layers.Conv1D(32, kernel_size=5, padding="same", activation="relu")(x)
# x = layers.Conv1D(64, kernel_size=5, padding="same", activation="relu")(x)
# x = layers.Conv1D(128, kernel_size=5, padding="same", activation="relu")(x)
# x = layers.Flatten()(x) # condense the output of our feature vector
# outputs = layers.Dense(46, activation="softmax")(x)
# model_USE= tf.keras.Model(inputs, outputs, name="USE")

model_USE = tf.keras.Sequential([
  sentence_encoder_layer, # take in sentences and then encode them into an embedding
  layers.Dense(64, activation="relu"),
  layers.Dense(46, activation="sigmoid")
], name="model_USE")

# Compile
model_USE.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_USE.summary()

Model: "model_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_4 (Dense)             (None, 64)                32832     
                                                                 
 dense_5 (Dense)             (None, 46)                2990      
                                                                 
Total params: 256,833,646
Trainable params: 35,822
Non-trainable params: 256,797,824
_________________________________________________________________


In [38]:
start = datetime.now()
model_USE_history = model_USE.fit(train_decode,
                                  one_hot_train_labels,
                                  epochs=10,
                                  validation_data=[test_decode,one_hot_test_labels],
                                  callbacks=[early_stopping, reduce_lr])

end = datetime.now()
print(f"The time taken to fit the modle is {end - start}")
model_USE.evaluate(test_decode,one_hot_test_labels)

Epoch 1/10
281/281 [==============================] - 7s 24ms/step - loss: 0.6695 - accuracy: 0.8367 - val_loss: 0.8375 - val_accuracy: 0.7921 - lr: 0.0010
Epoch 2/10
281/281 [==============================] - 7s 23ms/step - loss: 0.6493 - accuracy: 0.8457 - val_loss: 0.8218 - val_accuracy: 0.7996 - lr: 0.0010
Epoch 3/10
281/281 [==============================] - 6s 23ms/step - loss: 0.6288 - accuracy: 0.8477 - val_loss: 0.8223 - val_accuracy: 0.7947 - lr: 0.0010
Epoch 4/10
281/281 [==============================] - 6s 23ms/step - loss: 0.6137 - accuracy: 0.8515 - val_loss: 0.8207 - val_accuracy: 0.8001 - lr: 0.0010
Epoch 5/10
281/281 [==============================] - 6s 23ms/step - loss: 0.5977 - accuracy: 0.8552 - val_loss: 0.8137 - val_accuracy: 0.7965 - lr: 0.0010
Epoch 6/10
281/281 [==============================] - 6s 23ms/step - loss: 0.5828 - accuracy: 0.8570 - val_loss: 0.8052 - val_accuracy: 0.8032 - lr: 0.0010
Epoch 7/10
281/281 [==============================] - 6s 23ms/st

[0.8021830916404724, 0.8058770895004272]